In [2]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

"""
When there's too little data for the task to train a full scale model from scratch
1. take layers from a previously trained model
2. freeze the layers 
3. add new trainable layers ontop of the frozen layers
4. train the new layers on the dataset 
"""

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"], # split for training, validation and test
    as_supervised=True,  # Include labels
)

# standardise the sizes to 150x150
size = (150, 150)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

batch_size = 32
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10) # optimise loading speed
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

# randomly flip or rotate images to improve sample diversity and reduce overfitting
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),])

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False # Freeze the base_model

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# train
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 5
history = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

2023-06-01 09:48:30.381597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5


2023-06-01 09:48:42.513642: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [4]
	 [[{{node Placeholder/_3}}]]
2023-06-01 09:48:42.514187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [4]
	 [[{{node Placeholder/_2}}]]


144/291 [=============>................] - ETA: 1:04 - loss: 0.2186 - binary_accuracy: 0.9030

Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


...

277/291 [===========================>..] - ETA: 6s - loss: 0.1798 - binary_accuracy: 0.9216

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


291/291 [==============================] - ETA: 0s - loss: 0.1780 - binary_accuracy: 0.9227

2023-06-01 09:50:53.522329: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-01 09:50:53.522813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


291/291 [==============================] - 163s 551ms/step - loss: 0.1780 - binary_accuracy: 0.9227 - val_loss: 0.0831 - val_binary_accuracy: 0.9708
Epoch 2/5
291/291 [==============================] - 164s 563ms/step - loss: 0.1145 - binary_accuracy: 0.9526 - val_loss: 0.0776 - val_binary_accuracy: 0.9703
Epoch 3/5
291/291 [==============================] - 163s 560ms/step - loss: 0.1099 - binary_accuracy: 0.9551 - val_loss: 0.0744 - val_binary_accuracy: 0.9725
Epoch 4/5
291/291 [==============================] - 159s 545ms/step - loss: 0.1058 - binary_accuracy: 0.9554 - val_loss: 0.0792 - val_binary_accuracy: 0.9686
Epoch 5/5
291/291 [==============================] - 180s 618ms/step - loss: 0.1015 - binary_accuracy: 0.9579 - val_loss: 0.0694 - val_binary_accuracy: 0.9725


In [ ]:
base_model.trainable = True # unfreeze 

# optional fine-tuning: train the entire model with a low learning rate
# base model still running in inference mode since created with training=False 
# the batch normalization layers inside won't update
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

286/291 [============================>.] - ETA: 11s - loss: 0.0842 - binary_accuracy: 0.9658